In [5]:
import pandas as pd
import os

In [6]:
files = [f for f in sorted(os.listdir("../graph_data")) if not f.startswith(".~")]
files

['20210330_cmdb_ci_business_app_V2_noDescription.csv',
 '20210401-AccessIT-APPLICATIONS-ORGANIZATIONS-reduced_CMDB_exact_matches.csv',
 '20210511_cmdb_similarities_stricter.csv',
 '20210517-CMDB-AHD-hits.csv',
 '20210701_OS_InstanceMonthyCashOut.csv',
 '20210719_cmdb_similarities_sentencebert_08_threshold_conntected_components.csv',
 '20210719_cmdb_similarities_sentencebert_all.csv',
 '20210719_cmdb_similarities_sentencebert_max.csv',
 'CMDB_ALL_040521_Tab_1.csv']

In [11]:
def read_with_nulls(filepath, skiprows=None):
    df = pd.read_csv(filepath, sep=",", skiprows=skiprows).fillna("")
    return df

In [12]:
apps = read_with_nulls(f"../graph_data/{files[0]}")
apps.head()

,PERSID,Name,Number,Applicationtype,Applicationfamily,Architecturetype,Installtype,Userbase,Platform,Businesscriticality,...,GDPRProtectionLevel,Segment,ProductFamily,Product,LifecycleDecision,BusinessScore,TechnicalScore,TIMEQuadrant,LifecycleDecisionTiming-Quarter,LifecycleDecisionTiming-Year
0,E4FFAE5299FA01F7ADC0E4115B9A6514,ServiceNow-1,**********,,,,,,,,...,,*******************************************,**************************************,****************************************,**********************************************...,****,****,*******,**,****
1,456E85108E2501F7AB5DE4115B9A6514,ServiceNow-2,**********,**********,,,**********,,,,...,,*******************************************,*******************,***************************,,,,,,
2,90DA34DA0B3F01F885D7005056B82CEC,ServiceNow-3,**********,**********,,,**********,,,,...,,,,,,,,,,
3,93AC606FAFFB01F7BA8DE4115B9A6514,ServiceNow-4,**********,**********,,,**********,,,,...,,,,,,,,,,
4,F3E3F8C1C69E01FA84C0005056B8134A,ServiceNow-5,**********,,,,,,,,...,,,,,,****,*,*******,,


In [13]:
# Use persid list from connected components table for testing purposes
persid_list = [
    "0B47074F477801F79C2BE4115B9A6514",
    "9C93740E832601F79164E4115B9A6514",
    "D35DAED81E2801F9B65B005056B847E9",
    "1E0E01C7D50701F79118005056B85744",
    "5CAD0493DA0301F5A8AAE4115B9A6514",
]


In [18]:
os.makedirs("../graph_data_small", exist_ok=True)
apps.loc[apps["PERSID"].isin(persid_list)].to_csv(f"../graph_data_small/{files[0]}", index=False, header=True)

In [19]:
orgs = read_with_nulls(f"../graph_data/{files[1]}")
orgs["PERSID"] = orgs["PERSID"].str.replace("nr:", "")
orgs.head()

,PERSID,CMDB_Name,APPLICATION,SYSTEM,CLIENT,COMPANY_CODE,CODE,USERS
0,5CAD0493DA0301F5A8AAE4115B9A6514,AccessIT-1,*****************,******,,**,******,*
1,5CAD0493DA0301F5A8AAE4115B9A6514,AccessIT-2,*****************,******,,**,******,*
2,5CAD0493DA0301F5A8AAE4115B9A6514,AccessIT-3,*****************,******,,**,**********,*
3,5CAD0493DA0301F5A8AAE4115B9A6514,AccessIT-4,*****************,******,,**,******,*
4,5CAD0493DA0301F5A8AAE4115B9A6514,AccessIT-5,*****************,******,,**,********,*


In [20]:
orgs.iloc[:5, :].to_csv(f"../graph_data_small/{files[1]}", index=False, header=True)

## Read in similarity matrix and format as floats

In [21]:
def read_similarity_matrix(filepath):
    # Read in data correctly into columns
    similarities = (
        read_with_nulls(filepath)
        .reset_index(drop=True).T
        .reset_index().T
        .reset_index(drop=True)
    )
    # Get column headers correctly and format PERSID according to apps
    similarities.columns = ["PERSID", *similarities.columns[1:]]
    similarities["PERSID"] = similarities["PERSID"].str.replace("nr:", "")
    similarities.columns = ["PERSID", *similarities["PERSID"]]
    # Change all similarity column types to float
    similarity_matrix = similarities.loc[:, similarities.columns != 'PERSID'].astype(float)
    # Reconstruct similarity matrix with correct formatting and datatypes
    final_similarities = pd.concat((similarities["PERSID"], similarity_matrix), axis=1)
    return final_similarities

In [22]:
similarities_strict = read_similarity_matrix(f"../graph_data/{files[2]}")
similarities_strict.head()

,PERSID,E4FFAE5299FA01F7ADC0E4115B9A6514,456E85108E2501F7AB5DE4115B9A6514,90DA34DA0B3F01F885D7005056B82CEC,93AC606FAFFB01F7BA8DE4115B9A6514,F3E3F8C1C69E01FA84C0005056B8134A,8E8A6BF1515201F987F3005056B847E9,460F0D24269101F7AF8BE4115B9A6514,6A286136AA3C01FAB2CF005056BA2822,6AAD6F20AA3C01FA901B005056BA2822,...,F05CD222393D01FA8699005056B82CEC,54468CE7681301F99813005056B847E9,86F5ED061A3A01F99C37005056B86998,7F5A8B63B53801FA9396005056BA2822,7FBEE0A5B53801FA8E76005056BA2822,6A27245897C001F780CCE4115B9A6514,CA72D6543A1B01F6B394E4115B9A6514,04B3C532AB3201FABC0D005056BA2822,D56E090FF12901F5AF79E4115B9A6514,0518B09AAB3201FA957D005056BA2822
0,E4FFAE5299FA01F7ADC0E4115B9A6514,0.878279,-0.275834,0.457899,0.076679,-0.491748,0.870218,-0.739341,0.322957,0.500809,...,-0.355968,0.187947,0.456478,-0.974895,0.642312,0.916429,-0.205021,-0.200299,-0.017197,0.635161
1,456E85108E2501F7AB5DE4115B9A6514,-0.453970,-0.616793,0.701273,-0.774286,0.626247,0.975450,0.232609,0.842065,-0.316456,...,-0.182964,0.746499,0.328543,0.201800,0.354257,-0.533987,0.485813,0.685661,0.118786,0.445941
2,90DA34DA0B3F01F885D7005056B82CEC,0.837175,-0.903786,-0.404638,0.038540,0.167451,0.504112,0.978128,0.334137,-0.223711,...,0.176764,0.989042,0.777015,-0.889378,0.301161,0.956850,0.382354,0.675837,-0.653506,0.208568
3,93AC606FAFFB01F7BA8DE4115B9A6514,0.718245,-0.613304,0.709880,0.360724,-0.948991,0.534950,0.153313,0.607657,-0.417648,...,0.130364,-0.102079,-0.765602,-0.255401,-0.885640,0.450018,-0.320152,-0.229180,-0.124486,-0.950052
4,F3E3F8C1C69E01FA84C0005056B8134A,-0.314525,0.503899,-0.798859,-0.105932,-0.460995,0.223978,-0.298360,-0.540270,0.324437,...,-0.682356,0.556470,0.633029,0.724878,0.871210,-0.612976,0.685305,-0.924604,0.047776,-0.062155


In [23]:
(
    similarities_strict.loc[similarities_strict["PERSID"].isin(persid_list)].iloc[:, :6]
).to_csv(f"../graph_data_small/{files[2]}", index=False, header=True)


In [24]:
similarities_strict["PERSID"][0] == apps["PERSID"][0]
print(apps["PERSID"][0])

E4FFAE5299FA01F7ADC0E4115B9A6514


## Obtain AHD data with App PERSIDs

In [25]:
ahd_hits = read_with_nulls(f"../graph_data/{files[3]}")
ahd_hits["PERSID"] = ahd_hits["PERSID"].str.replace("nr:", "")

In [26]:
ahd_hits.loc[apps["PERSID"].isin(persid_list)].head()

,AHDhits,Name,PERSID
13,*,Ahd-14,0B47074F477801F79C2BE4115B9A6514
32,*,Ahd-33,5CAD0493DA0301F5A8AAE4115B9A6514
37,*,Ahd-38,9C93740E832601F79164E4115B9A6514
59,**,Ahd-60,1E0E01C7D50701F79118005056B85744
239,*,Ahd-240,D35DAED81E2801F9B65B005056B847E9


In [27]:
ahd_hits.loc[apps["PERSID"].isin(persid_list)].to_csv(f"../graph_data_small/{files[3]}", index=False, header=True)

## Obtain OS instances data

In [28]:
os_instances = read_with_nulls(f"../graph_data/{files[4]}")
# Rename first column as PERSID
os_instances.rename({u"os_\ufeffPersID": "PERSID"}, axis=1, inplace=True)
os_instances.head()

,PERSID,ServerName,Phase,Ongoingcosts€,OngoingAzurecostsBasicAHUB1yreserved€,OngoingAzureCostsManagedAHUB1yreserved€,SavingsinAzureAHUB1yreserved,Ipaddress,Class,Locationtype,...,OSSupport€,PowerConsumption€,AzureLift&Shift,AzureRightsizeAHUB,AzureRightsizeAHUB1yreserved,AzureRightsizedStorage,AzureNetwork,AzureStorageAmount,MSAzureComputeProfile,MSAzureRegion
0,F29988A1617D01F093F5001CC4EFEA68,Os-1,**********,******,,,,************,*****,*******************************,...,******,,,,,,****,,,
1,07FB5E84257E01FA8A87005056B86998,Os-2,**********,******,,,,************,*****,*******************************,...,******,,,,,,****,,,
2,09E8A010C2A701FAB769005056B86998,Os-3,**********,******,,,,***********,*****,*******************************,...,******,,,,,,****,,,
3,9C5D3D9C612001F989D3005056B82CEC,Os-4,,******,,,,************,*****,*******************************,...,******,,,,,,****,,,
4,3B002DB65CFE01F9A0C6005056B86998,Os-5,**********,******,,,,*************,*******,*******************************,...,******,,,,,,****,,,


In [29]:
os_instances.iloc[:5, :].to_csv(f"../graph_data_small/{files[4]}", index=False, header=True)

## Process similarity data for connected components

In [30]:
similarities_connected_components = read_with_nulls(f"../graph_data/{files[5]}")
similarities_connected_components.rename(
    {"PersID-1": "PERSID_1", "PersID-2": "PERSID_2"}, axis=1, inplace=True
)
similarities_connected_components["PERSID_2"] = similarities_connected_components[
    "PERSID_2"
].str.replace("nr:", "")
similarities_connected_components.head()


,PERSID_1,PERSID_2,similaritybertcomp,CompID
0,0B47074F477801F79C2BE4115B9A6514,81D25E08477801F78D47E4115B9A6514,0.922533,0
1,9C93740E832601F79164E4115B9A6514,ACD4EF419A0D01F79E7AE4115B9A6514,0.872425,1
2,04B009E37E3501F6A1F5E4115B9A6514,D252EB32BDF301F6BDCBE4115B9A6514,0.854366,2
3,1E0E01C7D50701F79118005056B85744,BB5CC23CD50B01F7A402005056B85744,0.840794,3
4,D6AEEF0C72C201F898A0005056B85744,6846CC724F7F01F794A2E4115B9A6514,0.906533,4


In [31]:
similarities_connected_components.iloc[:5, :].to_csv(f"../graph_data_small/{files[5]}", index=False, header=True)

In [32]:
similarities_all = read_similarity_matrix(f"../graph_data/{files[6]}")
similarities_all.head()

,PERSID,E4FFAE5299FA01F7ADC0E4115B9A6514,456E85108E2501F7AB5DE4115B9A6514,90DA34DA0B3F01F885D7005056B82CEC,93AC606FAFFB01F7BA8DE4115B9A6514,F3E3F8C1C69E01FA84C0005056B8134A,8E8A6BF1515201F987F3005056B847E9,460F0D24269101F7AF8BE4115B9A6514,6A286136AA3C01FAB2CF005056BA2822,6AAD6F20AA3C01FA901B005056BA2822,...,F05CD222393D01FA8699005056B82CEC,54468CE7681301F99813005056B847E9,86F5ED061A3A01F99C37005056B86998,7F5A8B63B53801FA9396005056BA2822,7FBEE0A5B53801FA8E76005056BA2822,6A27245897C001F780CCE4115B9A6514,CA72D6543A1B01F6B394E4115B9A6514,04B3C532AB3201FABC0D005056BA2822,D56E090FF12901F5AF79E4115B9A6514,0518B09AAB3201FA957D005056BA2822
0,E4FFAE5299FA01F7ADC0E4115B9A6514,0.260010,0.975300,0.757471,-0.108593,0.481735,0.827409,0.857941,0.031586,-0.391470,...,-0.100947,-0.713361,0.905804,-0.161224,-0.123305,0.863372,0.283273,0.968688,-0.590600,-0.685026
1,456E85108E2501F7AB5DE4115B9A6514,-0.934789,0.649085,0.309804,0.136988,-0.081008,0.711887,-0.801908,-0.737174,-0.052717,...,0.302369,-0.311928,0.295651,-0.423385,-0.016539,0.180955,0.148554,0.920161,0.334029,0.888974
2,90DA34DA0B3F01F885D7005056B82CEC,0.539601,0.906933,0.352781,-0.430481,0.786526,0.458938,0.917927,-0.550351,0.414944,...,0.115247,-0.657316,-0.742150,-0.494291,0.570340,-0.858648,0.933507,-0.171077,-0.273613,-0.323554
3,93AC606FAFFB01F7BA8DE4115B9A6514,-0.144373,0.948103,0.889477,-0.284909,-0.080668,0.628034,0.283471,0.699853,-0.403170,...,0.878891,-0.990040,-0.588508,0.602610,0.509574,0.142858,-0.441874,0.450621,0.274434,0.424639
4,F3E3F8C1C69E01FA84C0005056B8134A,0.474827,-0.057613,0.735571,-0.290776,0.570876,-0.543730,-0.879623,-0.334432,-0.679682,...,-0.278047,0.614581,0.481744,0.654260,0.633501,0.568810,-0.592600,0.726942,0.873346,-0.667437


In [33]:
(
    similarities_all.loc[similarities_all["PERSID"].isin(persid_list)].iloc[:, :6]
).to_csv(f"../graph_data_small/{files[6]}", index=False, header=True)

In [34]:
similarities_max = read_with_nulls(f"../graph_data/{files[7]}")
similarities_max.rename(
    {"PersID-1": "PERSID_1", "PersID-2": "PERSID_2"}, axis=1, inplace=True
)
similarities_max["PERSID_2"] = similarities_max[
    "PERSID_2"
].str.replace("nr:", "")
similarities_max.head()

,PERSID_1,PERSID_2,similaritybertmax
0,E4FFAE5299FA01F7ADC0E4115B9A6514,E4FFAE5299FA01F7ADC0E4115B9A6514,0.863724
1,456E85108E2501F7AB5DE4115B9A6514,456E85108E2501F7AB5DE4115B9A6514,0.953897
2,90DA34DA0B3F01F885D7005056B82CEC,90DA34DA0B3F01F885D7005056B82CEC,0.993023
3,93AC606FAFFB01F7BA8DE4115B9A6514,93AC606FAFFB01F7BA8DE4115B9A6514,0.921175
4,F3E3F8C1C69E01FA84C0005056B8134A,F3E3F8C1C69E01FA84C0005056B8134A,0.991696


In [35]:
similarities_max.loc[similarities_max["PERSID_1"].isin(persid_list)].to_csv(f"../graph_data_small/{files[7]}", index=False, header=True)

In [36]:
db_all = read_with_nulls(f"../graph_data/{files[8]}")
len(db_all)

/tmp/ipykernel_20422/4038377858.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,22,23,27,30,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  db_all = read_with_nulls(f"../graph_data/{files[8]}")


328475

In [37]:
db_all.columns

Index(['EAR_APP_ID', 'EAR_APP_NAME', 'EAR_OBJECTSTATE', 'ORG_NAME',
       'ALOCATION', 'PROCESSLEVEL', 'PROCESS_NAME', 'EAR_STARTDATE',
       'EAR_ENDDATE', 'EAR_DESCRIPTION', 'EAR_CAPABILITY',
       'EAR_CUSTOMIZATIONDEGREE', 'EAR_CLOUDTYPE', 'EAR_GDPRRELEVANT',
       'EAR_GXP', 'EAR_MAPPED_CMDB_ID', 'EAR_ROLES', 'CMDB_CI_NAME',
       'CMDB_CI_CLASS', 'CMDB_CI_DESCRIPTION', 'CMDB_CI_STATUS',
       'CMDB_CI_PERSID', 'CMDB_CI_ID_OF_PARENT_CI', 'CMDB_CI_TREE_LEVEL_ID',
       'CMDB_CI_FAMILY', 'CMDB_CI_CLASS2', 'IP', 'Servername', 'DNSName',
       'BusinessApplicationCMDB', 'BASubclassCMDB', 'Region', 'CountryName',
       'City', 'FRC', 'FDAGMP', 'BusinessContact', 'OrgCodeBC', 'OrgCodeSC',
       'ServiceContact', 'BATechnicalContact', 'OrgCodeBATC',
       'TechnicalContact', 'TCOrgCode', 'ApplicationOwner',
       'ApplicationOwnerOrgCode', 'SecurityClassification', 'Operatedfor',
       'Environment'],
      dtype='object')

In [38]:
def clean_strings(df, colnames=[]):
    for col in colnames:
        df[col] = df[col].str.replace("nr:", "")
    return df

In [39]:
db_all_clean = clean_strings(db_all, colnames=["EAR_MAPPED_CMDB_ID", "CMDB_CI_PERSID", "CMDB_CI_ID_OF_PARENT_CI"])

In [40]:
(
    db_all_clean.loc[db_all_clean["EAR_MAPPED_CMDB_ID"].isin(persid_list)]
).to_csv(f"../graph_data_small/{files[8]}", index=False, header=True)

In [41]:
# cond1 = os_instances.loc[os_instances["PERSID"].isin(db_all_clean["CMDB_CI_PERSID"])]
# os_instances.loc[cond1.index]

In [42]:
# cond2 = apps.loc[apps["PERSID"].isin(db_all_clean["CMDB_CI_PERSID"])]
# apps.loc[cond2.index]

In [43]:
db_all_clean.head()

,EAR_APP_ID,EAR_APP_NAME,EAR_OBJECTSTATE,ORG_NAME,ALOCATION,PROCESSLEVEL,PROCESS_NAME,EAR_STARTDATE,EAR_ENDDATE,EAR_DESCRIPTION,...,ServiceContact,BATechnicalContact,OrgCodeBATC,TechnicalContact,TCOrgCode,ApplicationOwner,ApplicationOwnerOrgCode,SecurityClassification,Operatedfor,Environment
0,APP-7167,****************************,******,,,,,**********,**********,****************************,...,,,,,,,,,,
1,APP-857,*****,******,,,,,**********,**********,**********************************************...,...,,,,,,,,,,
2,APP-857,*****,******,,,,,**********,**********,**********************************************...,...,,,,,,,,,,
3,APP-857,*****,******,,,,,**********,**********,**********************************************...,...,,,,,,,,,,
4,APP-857,*****,******,,,,,**********,**********,**********************************************...,...,**********************,**********************,******,*****************************,******,,,************,***************************,****


## Check for presence of IDs in the applications and organizations CSVs

In [44]:
any(db_all_clean["CMDB_CI_PERSID"].isin(orgs["PERSID"]))

True

In [45]:
any(db_all_clean["CMDB_CI_ID_OF_PARENT_CI"].isin(orgs["PERSID"]))

True

In [46]:
any(db_all_clean["CMDB_CI_PERSID"].isin(apps["PERSID"]))

True

In [47]:
any(db_all_clean["CMDB_CI_ID_OF_PARENT_CI"].isin(apps["PERSID"]))

True

In [48]:
any(db_all_clean["CMDB_CI_PERSID"].isin(os_instances["PERSID"]))

True

In [49]:
any(db_all_clean["CMDB_CI_ID_OF_PARENT_CI"].isin(os_instances["PERSID"]))

True

## Observations
The `CMDB_ALL_040521_Tab_1.csv` file contains data from apps, orgs and OS instances PERSIDs, all in the two columns `CMDB_CI_PERSID` and `CMDB_CI_ID_OF_PARENT_CI`.